# 第25章：综合实战项目

综合运用所学知识,完成实战项目,将理论转化为实践能力。

## 什么是实战项目?

**实战项目就是将所学知识综合运用,开发一个完整的、可运行的程序。**

### 本章项目

我们将完成5个实战项目:
1. **学生管理系统** - 数据管理、文件操作
2. **待办事项应用** - 时间管理、任务追踪
3. **记账本程序** - 财务管理、数据分析
4. **简易博客系统** - 内容管理、搜索功能
5. **文件整理工具** - 自动化、批量处理

## 项目1：学生管理系统

一个完整的学生信息管理系统,包含增删改查、成绩管理、数据持久化等功能。

In [ ]:
import json
import csv
import os
from datetime import datetime

class Student:
    """学生类"""
    def __init__(self, name, student_id, age, gender):
        self.name = name
        self.student_id = student_id
        self.age = age
        self.gender = gender
        self.scores = {}
        self.created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    def add_score(self, subject, score):
        if not 0 <= score <= 100:
            raise ValueError("分数必须在0-100之间")
        self.scores[subject] = score
    
    def get_average(self):
        if not self.scores:
            return 0
        return sum(self.scores.values()) / len(self.scores)
    
    def get_total(self):
        return sum(self.scores.values())
    
    def to_dict(self):
        return {
            "name": self.name,
            "student_id": self.student_id,
            "age": self.age,
            "gender": self.gender,
            "scores": self.scores,
            "created_at": self.created_at
        }
    
    @classmethod
    def from_dict(cls, data):
        student = cls(data["name"], data["student_id"], data["age"], data["gender"])
        student.scores = data.get("scores", {})
        student.created_at = data.get("created_at", "")
        return student

class StudentManager:
    """学生管理系统"""
    def __init__(self, filename="students.json"):
        self.filename = filename
        self.students = []
        self.load_data()
    
    def load_data(self):
        if os.path.exists(self.filename):
            try:
                with open(self.filename, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    self.students = [Student.from_dict(s) for s in data]
                print(f"成功加载 {len(self.students)} 个学生记录")
            except Exception as e:
                print(f"加载失败: {e}")
                self.students = []
        else:
            print("创建新系统")
            self.students = []
    
    def save_data(self):
        try:
            data = [s.to_dict() for s in self.students]
            with open(self.filename, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            return True
        except Exception as e:
            print(f"保存失败: {e}")
            return False
    
    def add_student(self, name, student_id, age, gender):
        student = Student(name, student_id, age, gender)
        self.students.append(student)
        if self.save_data():
            print(f"学生 {name} 添加成功")
            return True
        return False
    
    def get_statistics(self):
        if not self.students:
            print("暂无学生数据")
            return
        total = len(self.students)
        male = sum(1 for s in self.students if s.gender == "男")
        female = total - male
        print(f"总人数: {total}, 男生: {male}, 女生: {female}")

# 使用示例
manager = StudentManager()
manager.add_student("张三", "2024001", 18, "男")
manager.add_student("李四", "2024002", 19, "女")
manager.get_statistics()

## 项目2：待办事项应用

功能完整的任务管理应用,支持任务分类、优先级、提醒等功能。

In [ ]:
from datetime import datetime, timedelta
import json
import os

class Task:
    """任务类"""
    def __init__(self, title, description="", priority="medium", category="general", due_date=None):
        self.id = None
        self.title = title
        self.description = description
        self.priority = priority
        self.category = category
        self.completed = False
        self.created_at = datetime.now()
        self.due_date = due_date
        self.completed_at = None
    
    def complete(self):
        self.completed = True
        self.completed_at = datetime.now()
    
    def is_overdue(self):
        if not self.due_date or self.completed:
            return False
        return datetime.now() > self.due_date
    
    def to_dict(self):
        return {
            "id": self.id,
            "title": self.title,
            "description": self.description,
            "priority": self.priority,
            "category": self.category,
            "completed": self.completed,
            "created_at": self.created_at.isoformat(),
            "due_date": self.due_date.isoformat() if self.due_date else None,
            "completed_at": self.completed_at.isoformat() if self.completed_at else None
        }
    
    @classmethod
    def from_dict(cls, data):
        task = cls(
            data["title"],
            data.get("description", ""),
            data.get("priority", "medium"),
            data.get("category", "general"),
            datetime.fromisoformat(data["due_date"]) if data.get("due_date") else None
        )
        task.id = data.get("id")
        task.completed = data.get("completed", False)
        task.created_at = datetime.fromisoformat(data["created_at"])
        if data.get("completed_at"):
            task.completed_at = datetime.fromisoformat(data["completed_at"])
        return task

class TodoApp:
    """待办事项应用"""
    def __init__(self, filename="todos.json"):
        self.filename = filename
        self.tasks = []
        self.next_id = 1
        self.load()
    
    def load(self):
        if os.path.exists(self.filename):
            try:
                with open(self.filename, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    self.tasks = [Task.from_dict(t) for t in data["tasks"]]
                    self.next_id = data.get("next_id", 1)
                print(f"成功加载 {len(self.tasks)} 个任务")
            except Exception as e:
                print(f"加载失败: {e}")
    
    def save(self):
        try:
            data = {
                "tasks": [t.to_dict() for t in self.tasks],
                "next_id": self.next_id
            }
            with open(self.filename, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            return True
        except Exception as e:
            print(f"保存失败: {e}")
            return False
    
    def add_task(self, title, description="", priority="medium", category="general", due_days=None):
        due_date = None
        if due_days is not None:
            due_date = datetime.now() + timedelta(days=due_days)
        task = Task(title, description, priority, category, due_date)
        task.id = self.next_id
        self.next_id += 1
        self.tasks.append(task)
        if self.save():
            print(f"任务已添加: {task.title}")
            return task
        return None

# 使用示例
app = TodoApp()
app.add_task("完成Python教程", "学完第25章", "high", "学习", 7)
app.add_task("买菜", "番茄、土豆", "medium", "生活", 1)
print(f"当前任务数: {len(app.tasks)}")

## 项目3：记账本程序

功能完整的个人财务管理工具,支持收支记录、分类统计、预算管理等。

In [ ]:
from datetime import datetime, timedelta
import csv
import json
from collections import defaultdict

class ExpenseTracker:
    """记账本程序"""
    def __init__(self, csv_file="expenses.csv", budget_file="budget.json"):
        self.csv_file = csv_file
        self.budget_file = budget_file
        self.expenses = []
        self.budgets = {}
        self.load_expenses()
        self.load_budgets()
    
    def load_expenses(self):
        try:
            with open(self.csv_file, "r", encoding="utf-8") as f:
                reader = csv.DictReader(f)
                self.expenses = list(reader)
            print(f"成功加载 {len(self.expenses)} 条记录")
        except FileNotFoundError:
            print("创建新的记账本")
            self.expenses = []
    
    def save_expenses(self):
        if not self.expenses:
            return
        with open(self.csv_file, "w", newline="", encoding="utf-8-sig") as f:
            fieldnames = ["日期", "类型", "分类", "金额", "描述"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(self.expenses)
    
    def load_budgets(self):
        try:
            with open(self.budget_file, "r", encoding="utf-8") as f:
                self.budgets = json.load(f)
        except FileNotFoundError:
            self.budgets = {}
    
    def save_budgets(self):
        with open(self.budget_file, "w", encoding="utf-8") as f:
            json.dump(self.budgets, f, ensure_ascii=False, indent=2)
    
    def add_expense(self, amount, category, description, expense_type="支出"):
        record = {
            "日期": datetime.now().strftime("%Y-%m-%d"),
            "类型": expense_type,
            "分类": category,
            "金额": str(amount),
            "描述": description
        }
        self.expenses.append(record)
        self.save_expenses()
        type_text = "收入" if expense_type == "收入" else "支出"
        print(f"{type_text}记录已添加: {category} ¥{amount}")
    
    def get_summary(self):
        if not self.expenses:
            print("没有记录")
            return
        income_total = 0
        expense_total = 0
        for record in self.expenses:
            amount = float(record["金额"])
            if record["类型"] == "收入":
                income_total += amount
            else:
                expense_total += amount
        print(f"总收入: ¥{income_total:.2f}")
        print(f"总支出: ¥{expense_total:.2f}")
        print(f"结余: ¥{income_total - expense_total:.2f}")
    
    def set_budget(self, category, amount):
        self.budgets[category] = amount
        self.save_budgets()
        print(f"已设置 {category} 预算: ¥{amount}")

# 使用示例
tracker = ExpenseTracker()
tracker.add_expense(50, "餐饮", "午餐")
tracker.add_expense(5000, "工资", "本月工资", "收入")
tracker.set_budget("餐饮", 1000)
tracker.get_summary()

## 项目4：简易博客系统

支持文章发布、标签管理、搜索等功能的博客系统。

In [ ]:
import json
from datetime import datetime
import os

class Post:
    """文章类"""
    def __init__(self, title, content, tags=None):
        self.id = None
        self.title = title
        self.content = content
        self.tags = tags or []
        self.views = 0
        self.created_at = datetime.now()
        self.updated_at = datetime.now()
    
    def add_view(self):
        self.views += 1
    
    def to_dict(self):
        return {
            "id": self.id,
            "title": self.title,
            "content": self.content,
            "tags": self.tags,
            "views": self.views,
            "created_at": self.created_at.isoformat(),
            "updated_at": self.updated_at.isoformat()
        }
    
    @classmethod
    def from_dict(cls, data):
        post = cls(data["title"], data["content"], data.get("tags", []))
        post.id = data.get("id")
        post.views = data.get("views", 0)
        post.created_at = datetime.fromisoformat(data["created_at"])
        post.updated_at = datetime.fromisoformat(data["updated_at"])
        return post

class Blog:
    """博客系统"""
    def __init__(self, filename="blog.json"):
        self.filename = filename
        self.posts = []
        self.next_id = 1
        self.load()
    
    def load(self):
        if os.path.exists(self.filename):
            try:
                with open(self.filename, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    self.posts = [Post.from_dict(p) for p in data["posts"]]
                    self.next_id = data.get("next_id", 1)
                print(f"成功加载 {len(self.posts)} 篇文章")
            except Exception as e:
                print(f"加载失败: {e}")
    
    def save(self):
        try:
            data = {
                "posts": [p.to_dict() for p in self.posts],
                "next_id": self.next_id
            }
            with open(self.filename, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            return True
        except Exception as e:
            print(f"保存失败: {e}")
            return False
    
    def create_post(self, title, content, tags=None):
        post = Post(title, content, tags)
        post.id = self.next_id
        self.next_id += 1
        self.posts.append(post)
        if self.save():
            print(f"文章已发布: {title}")
            return post.id
        return None
    
    def search_posts(self, keyword):
        keyword = keyword.lower()
        results = []
        for post in self.posts:
            if (keyword in post.title.lower() or
                keyword in post.content.lower() or
                any(keyword in tag.lower() for tag in post.tags)):
                results.append(post)
        return results
    
    def get_statistics(self):
        if not self.posts:
            print("暂无文章")
            return
        total_posts = len(self.posts)
        total_views = sum(p.views for p in self.posts)
        print(f"文章总数: {total_posts}")
        print(f"总浏览量: {total_views}")
        print(f"平均浏览: {total_views/total_posts:.1f}")

# 使用示例
blog = Blog()
blog.create_post("Python入门指南", "Python是一门易学易用的编程语言...", ["Python", "编程"])
blog.create_post("数据结构与算法", "学习数据结构很重要...", ["算法", "数据结构"])
blog.get_statistics()

## 项目5：文件整理工具

自动整理文件的实用工具,按类型、日期等规则归类文件。

In [ ]:
from pathlib import Path
import shutil
from datetime import datetime

class FileOrganizer:
    """文件整理工具"""
    def __init__(self, source_dir):
        self.source_dir = Path(source_dir)
        self.file_types = {
            "图片": [".jpg", ".jpeg", ".png", ".gif", ".bmp"],
            "文档": [".txt", ".doc", ".docx", ".pdf", ".xlsx"],
            "视频": [".mp4", ".avi", ".mkv", ".mov"],
            "音频": [".mp3", ".wav", ".flac"],
            "压缩包": [".zip", ".rar", ".7z", ".tar"]
        }
    
    def _get_category(self, suffix):
        for category, extensions in self.file_types.items():
            if suffix in extensions:
                return category
        return None
    
    def organize_by_type(self, dry_run=False):
        """按类型整理文件"""
        if not self.source_dir.exists():
            print(f"目录不存在: {self.source_dir}")
            return
        
        print(f"开始整理: {self.source_dir}")
        print(f"模式: {'预览' if dry_run else '执行'}")
        
        if not dry_run:
            for category in self.file_types.keys():
                category_dir = self.source_dir / category
                category_dir.mkdir(exist_ok=True)
        
        moved_count = 0
        for file in self.source_dir.iterdir():
            if file.is_dir():
                continue
            category = self._get_category(file.suffix.lower())
            if category:
                dest = self.source_dir / category / file.name
                action = "将移动" if dry_run else "移动"
                print(f"{action}: {file.name} -> {category}/")
                if not dry_run:
                    shutil.move(str(file), str(dest))
                moved_count += 1
        
        print(f"完成! 共处理 {moved_count} 个文件")
    
    def get_statistics(self):
        """获取统计信息"""
        print(f"文件统计: {self.source_dir}")
        total_files = 0
        total_size = 0
        by_type = {}
        
        for file in self.source_dir.rglob("*"):
            if file.is_file():
                total_files += 1
                size = file.stat().st_size
                total_size += size
                category = self._get_category(file.suffix.lower())
                if category:
                    if category not in by_type:
                        by_type[category] = {"count": 0, "size": 0}
                    by_type[category]["count"] += 1
                    by_type[category]["size"] += size
        
        print(f"总文件数: {total_files}")
        print(f"总大小: {total_size / (1024**3):.2f} GB")
        
        if by_type:
            print("\n按类型统计:")
            for category, data in sorted(by_type.items()):
                size_mb = data["size"] / (1024**2)
                print(f"{category}: {data['count']} 个, {size_mb:.2f} MB")

# 使用示例
# organizer = FileOrganizer(".")
# organizer.get_statistics()
# organizer.organize_by_type(dry_run=True)  # 预览模式
print("文件整理工具已定义完成")

## 总结

恭喜你完成了Python入门教程的所有内容！通过这5个实战项目,你已经掌握了:

- ✅ 数据管理和持久化(JSON/CSV)
- ✅ 面向对象编程
- ✅ 文件操作
- ✅ 异常处理
- ✅ 数据结构应用
- ✅ 项目开发流程

### 下一步学习方向

1. **Web开发** - Flask/Django
2. **数据分析** - Pandas/NumPy
3. **机器学习** - Scikit-learn
4. **自动化** - Selenium/PyAutoGUI
5. **爬虫** - Requests/BeautifulSoup

**继续前进,编程之路无限精彩!** 🚀